In [1]:
import os
import calendar
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean
import csv
import matplotlib.colors as colors
import cartopy.feature as cfeature 
import matplotlib.ticker as mticker
import cartopy.mpl.ticker as cartopyticker
coast = cfeature.GSHHSFeature(scale="full")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
plt.rcParams.update({'font.size': 14})

In [2]:
mycolor = "#6667AB"
mycolor2 = "#456A30" # Green treetop
dom = [-20.5, 11.75, 41.25, 67.]
mainproj = ccrs.Mercator(central_longitude=0.5*(dom[0] + dom[1]),
    min_latitude=dom[2], max_latitude=dom[3])
datacrs = ccrs.PlateCarree();

In [4]:
datadir = "../data/"
figdir = "../figures/L05/"
resdir = "../results/L05/"
datafile = os.path.join(datadir, "MBA_CPRdata_Emodnet_21Jan22.csv")
woafile = "../data/WOD/temperature_surface_WOD2.nc"
bathfile = os.path.join(datadir, "gebco_30sec_16.nc")
outputfile1 = os.path.join(resdir, "calanus_finmarchicus_year_month.nc")
outputfile2 = os.path.join(resdir, "calanus_helgolandicus_year_month.nc")

## Read WOD data

In [10]:
with netCDF4.Dataset(woafile, "r") as nc:
    print(nc)
    lonWOA = nc.get_variables_by_attributes(standard_name="longitude")[0][:]
    latWOA = nc.get_variables_by_attributes(standard_name="latitude")[0][:]
    time = nc.get_variables_by_attributes(standard_name="time")[0]
    T = nc.variables["Temperature"][:]
    datesWOA = netCDF4.num2date(time[:], time.units, only_use_cftime_datetimes=False)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): observations(529985), idlen(14)
    variables(dimensions): float32 obslon(observations), float32 obslat(observations), float64 obstime(observations), float32 obsdepth(observations), |S1 obsid(observations, idlen), float32 Temperature(observations)
    groups: 


## Load observations

In [26]:
def read_data_csv(datafile):
    lon = []
    lat = []
    year = []
    month = []
    calanusfinmarchicus = []
    calanushelgolandicus = []
    with open(datafile, "r") as df:
        df.readline()
        for lines in df:
            lsplit = lines.rstrip().split(',')
            lat.append(float(lsplit[1]))
            lon.append(float(lsplit[2]))
            year.append(int(lsplit[3]))
            month.append(int(lsplit[4]))
            calanusfinmarchicus.append(float(lsplit[5]))
            calanushelgolandicus.append(float(lsplit[6]))
            
    return np.array(lon), np.array(lat), np.array(year), np.array(month), np.array(calanusfinmarchicus), np.array(calanushelgolandicus)

In [27]:
lonobs, latobs, yearobs, monthobs, calanusfinmarchicus, calanushelgolandicus = read_data_csv(datafile)

## Load results

In [13]:
def get_results(outputfile):
    with netCDF4.Dataset(outputfile) as nc:
        lon = nc.get_variables_by_attributes(standard_name="longitude")[0][:]
        lat = nc.get_variables_by_attributes(standard_name="latitude")[0][:]
        time = nc.get_variables_by_attributes(standard_name="time")[0]
        dates = netCDF4.num2date(time[:], time.units, only_use_cftime_datetimes=False)
        field = nc.variables["abundance"][:]
        error = nc.variables["error"][:]
        
    return lon, lat, dates, field, error

In [14]:
lon1, lat1, dates1, field1, error1 = get_results(outputfile1)
_, _, _, field2, error2 = get_results(outputfile2)

In [15]:
ntimes = len(dates1)
for ii in range(490, 510):
    
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111, projection=mainproj)
    pcm = ax.pcolormesh(lon1, lat1, field1[ii,:,:],
                       cmap=plt.cm.hot_r, vmin=0., vmax=25., transform=datacrs)

    cb = plt.colorbar(pcm, extend="max")

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
    ax.set_extent(dom)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title(f"Canalus finmarchicus, {calendar.month_name[dates1[ii].month]} {dates1[ii].year}")
    
    figname = f"Canalus_finmarchicus_L05_{dates1[ii].year}{str(dates1[ii].month).zfill(2)}.jpg"
    plt.savefig(os.path.join(figdir, figname), dpi=300, bbox_inches="tight")
    plt.close()

In [16]:
for ii in range(490, 510):
    
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111, projection=mainproj)
    pcm = ax.pcolormesh(lon1, lat1, field2[ii,:,:],
                       cmap=plt.cm.hot_r, vmin=0., vmax=25., transform=datacrs)

    cb = plt.colorbar(pcm, extend="max")

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
    ax.set_extent(dom)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title(f"Canalus helgolandicus, {calendar.month_name[dates1[ii].month]} {dates1[ii].year}")
    
    figname = f"Canalus_helgolandicus_L05_{dates1[ii].year}{str(dates1[ii].month).zfill(2)}.jpg"
    plt.savefig(os.path.join(figdir, figname), dpi=300, bbox_inches="tight")
    plt.close()

In [17]:
for ii in range(490, 510):
    
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111, projection=mainproj)
    pcm = ax.pcolormesh(lon1, lat1, error1[ii,:,:],
                       cmap=plt.cm.RdYlGn_r, vmin=0., vmax=1., transform=datacrs)

    cb = plt.colorbar(pcm)

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
    ax.set_extent(dom)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title(f"Error field, {calendar.month_name[dates1[ii].month]} {dates1[ii].year}")
    
    figname = f"Canalus_finmarchicus_L05_error_{dates1[ii].year}{str(dates1[ii].month).zfill(2)}.jpg"
    plt.savefig(os.path.join(figdir, figname), dpi=300, bbox_inches="tight")
    plt.close()

In [29]:
for yyyy in range(2001, 2002):
    
    for mm in range(1, 5):
        
        goodtime = np.where((yearobs == yyyy) & (monthobs == mm))[0]
    
        fig = plt.figure(figsize=(12, 8))
        ax = plt.subplot(111, projection=mainproj)
        
        ax.plot(lonobs[goodtime], latobs[goodtime], "ko", ms=.25, transform=datacrs, zorder=3)
        sca = ax.scatter(lonobs[goodtime], latobs[goodtime], s=8, c=calanusfinmarchicus[goodtime],
                           cmap=plt.cm.hot_r, vmin=0., vmax=25., transform=datacrs, zorder=2)

        cb = plt.colorbar(sca, extend="max")

        ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
        ax.set_extent(dom)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                              linewidth=.5, color="gray", alpha=1, linestyle="--")
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = lon_formatter
        gl.yformatter = lat_formatter
        ax.set_title(f"Canalus finmarchicus observations, {calendar.month_name[mm]} {yyyy}")

        figname = f"Canalus_finmarchicus_obs_{yyyy}{str(mm).zfill(2)}.jpg"
        plt.savefig(os.path.join(figdir, figname), dpi=300, bbox_inches="tight")
        plt.close()

In [17]:
yearsWOA = np.array([dd.year for dd in datesWOA])
monthWOA = np.array([dd.month for dd in datesWOA])

In [24]:
for yyyy in range(2001, 2002):
    
    for mm in range(1, 13):
        
        goodtime = np.where((yearsWOA == yyyy) & (monthWOA == mm))[0]
    
        fig = plt.figure(figsize=(12, 8))
        ax = plt.subplot(111, projection=mainproj)
        sca = ax.scatter(lonWOA[goodtime], latWOA[goodtime], s=8, c=T[goodtime],
                           cmap=plt.cm.RdYlBu_r, vmin=2., vmax=15., transform=datacrs)

        cb = plt.colorbar(sca, extend="both")

        ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
        ax.set_extent(dom)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                              linewidth=.5, color="gray", alpha=1, linestyle="--")
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = lon_formatter
        gl.yformatter = lat_formatter
        ax.set_title(f"Temperature observations, {calendar.month_name[mm]} {yyyy}")

        figname = f"temperature_obs_{yyyy}{str(mm).zfill(2)}.jpg"
        plt.savefig(os.path.join(figdir, figname), dpi=300, bbox_inches="tight")
        plt.close()

In [22]:
np.max(T)

29.98